# Project 3 - Setup Graph DB from SQL DB of BART stations

University of California, Berkeley

Master of Information and Data Science (MIDS) program

w205 - Fundamentals of Data Engineering


# Included Modules and Packages

Import of interest here is the graph DB neo4j

In [1]:
import neo4j

import csv

import math
import numpy as np
import pandas as pd

import psycopg2

# Supporting code

Python code to facilitate loading the Neo4j DB and reviewing results.
Our queries are written in neo4j style (i.e., NoSQL) rather than SQL here.

In [2]:
driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))

In [3]:
session = driver.session(database="neo4j")

In [4]:
def my_neo4j_wipe_out_database():
    "wipe out database by deleting all nodes and relationships"
    
    query = "match (node)-[relationship]->() delete node, relationship"
    session.run(query)
    
    query = "match (node) delete node"
    session.run(query)

In [5]:
def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

In [6]:
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")


In [7]:
def my_neo4j_create_node(station_name):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Station {name: $station_name})
    
    """
    
    session.run(query, station_name=station_name)
    

In [8]:
def my_neo4j_create_relationship_one_way(from_station, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [9]:
def my_neo4j_create_relationship_two_way(from_station, to_station, weight):
    "create relationships two way between two stations with a weight"
    
    query = """
    
    MATCH (from:Station), 
          (to:Station)
    WHERE from.name = $from_station and to.name = $to_station
    CREATE (from)-[:LINK {weight: $weight}]->(to),
           (to)-[:LINK {weight: $weight}]->(from)
    
    """
    
    session.run(query, from_station=from_station, to_station=to_station, weight=weight)
    

In [10]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [11]:
cursor = connection.cursor()

# Introduction 

We will now go step by step through the process of creating a graph database in Neo4j for the BART sytem. 

We will use some of the queries in 3.2 to pull the needed data in the right format to create nodes and relationships in our Neo4j graph database

We will use the functions created above to create the nodes and relationships:
* my_neo4j_create_node() - creates a node with label Station
* my_neo4j_create_relationship_one_way() - creates a relationship one way between two stations with a weight
* my_neo4j_create_relationship_two_way() - create relationships two way between two stations with a weight

The way we create it might seem a bit strange at first. However, we want to be able to use the canned Neo4j Graph Data Science algorithms, and this design allows us to do so.

# Wipe Out and Verify Neo4j database

Ensure we're starting clean.

In [12]:
my_neo4j_wipe_out_database()
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 0
  Relationships: 0
-------------------------


# Query the list of stations and create the departure and arrival nodes in the graph

Use the query from 3.2.1 "Query the list of stations"

For each station X, create two nodes:
* depart X
* arrive X

Use the function my_neo4j_create_node() defined above

For example, West Oakland:
* my_neo4j_create_node('depart West Oakland')
* my_neo4j_create_node('arrive West Oakland')



In [13]:
connection.rollback()

query = """

select station
from stations
order by station

"""

cursor.execute(query)

connection.rollback()

# We get all the stations database rows back (50) with the station
# Each station is a node that you can depart or arrive from

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    
    my_neo4j_create_node('depart ' + station)
    my_neo4j_create_node('arrive ' + station)
    

# Verify the number of nodes and relationships

Since there are 50 stations, we should have 100 Nodes in the graph.
One is labeled departStation the other arriveStation


In [14]:
# Verify graph counts
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 100
  Relationships: 0
-------------------------


# For all stations, create line nodes, and create relationships between the line nodes and the departure and arrival nodes with weight 0

For each station X and each line Y that the station serves:
* Create a line node
* Create a relationship from the departure node to the line node with weight 0
* Create a relationship from the line node to the arrival node with weight 0

In [15]:
connection.rollback()

query = """

select station, line
from lines
order by station, line

"""

cursor.execute(query)

connection.rollback()

# The lines database has 114 rows
# Therefore our graph of 100 nodes grows by 114 to total 214
# We add 2 relationships for each of the 114 nodes so we get 228 relationships

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    line = row[1]
    
    depart = 'depart ' + station
    arrive = 'arrive ' + station
    line_station = line + ' ' + station
    
    my_neo4j_create_node(line_station)
    my_neo4j_create_relationship_one_way(depart, line_station, 0)
    my_neo4j_create_relationship_one_way(line_station, arrive, 0)
    

In [16]:
# Verify graph counts
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 228
-------------------------


# Query the list of all possible line transfers and the transfer times, create a relationship for each transfer with the transfer time as the weight

Query the list of all possible line transfers and the transfer times

For each station X, from line Y, to line Z, create a relationship from Y's line node to Z's line node with the weight set to the transfer time


In [17]:
connection.rollback()

query = """

select a.station, a.line as from_line, b.line as to_line, s.transfer_time
from lines a
     join lines b
       on a.station = b.station and a.line <> b.line 
     join stations s
       on a.station = s.station
order by 1, 2, 3

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    station = row[0]
    from_line = row[1]
    to_line = row[2]
    transfer_time = int(row[3])
    
    from_station = from_line + ' ' + station
    to_station = to_line + ' ' + station
    
    my_neo4j_create_relationship_one_way(from_station, to_station, transfer_time)


In [18]:
# Verify graph counts
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 436
-------------------------


# Query the list of all segments between each station and its adjoining stations, create a relationship for each segment both ways

Query the list of all segments between each station and its adjoining stations

For each segment from station X to station Y on line Z, create two relationships:
* From X's line node to Y's line node with travel time
* From Y's line node to X's line node with travel time


In [19]:
connection.rollback()

query = """

select a.line, a.station as from_station, b.station as to_station, t.travel_time
from lines a
  join lines b
    on a.line = b.line and b.sequence = (a.sequence + 1)
  join travel_times t
    on (a.station = t.station_1 and b.station = t.station_2)
        or (a.station = t.station_2 and b.station = t.station_1)
order by line, from_station, to_station

"""

cursor.execute(query)

connection.rollback()

rows = cursor.fetchall()

for row in rows:
    
    line = row[0]
    from_station = line + ' ' + row[1]
    to_station = line + ' ' + row[2]
    travel_time = int(row[3])
    
    my_neo4j_create_relationship_two_way(from_station, to_station, travel_time)
    

In [20]:
# Verify graph counts
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 214
  Relationships: 652
-------------------------


# Verify the graph we just built checking shortest paths


In [21]:
def my_neo4j_shortest_path(from_station, to_station):
    "given a from station and to station, run and print the shortest path"
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', 'Station', 'LINK', {relationshipProperties: 'weight'})"
    session.run(query)

    query = """

    MATCH (source:Station {name: $source}), (target:Station {name: $target})
    CALL gds.shortestPath.dijkstra.stream(
        'ds_graph', 
        { sourceNode: source, 
          targetNode: target, 
          relationshipWeightProperty: 'weight'
        }
    )
    YIELD index, sourceNode, targetNode, totalCost, nodeIds, costs, path
    RETURN
        gds.util.asNode(sourceNode).name AS from,
        gds.util.asNode(targetNode).name AS to,
        totalCost,
        [nodeId IN nodeIds | gds.util.asNode(nodeId).name] AS nodes,
        costs
    ORDER BY index

    """

    result = session.run(query, source=from_station, target=to_station)
    
    for r in result:
        
        total_cost = int(r['totalCost'])
        
        print("\n--------------------------------")
        print("   Total Cost: ", total_cost)
        print("   Minutes: ", round(total_cost / 60.0,1))
        print("--------------------------------")
        
        nodes = r['nodes']
        costs = r['costs']
        
        i = 0
        previous = 0
        
        for n in nodes:
            
            print(n + ", " + str(int(costs[i]) - previous)  + ", " + str(int(costs[i])))
            
            previous = int(costs[i])
            i += 1
    

In [22]:
my_neo4j_shortest_path('depart Dublin', 'arrive Antioch')


--------------------------------
   Total Cost:  5813
   Minutes:  96.9
--------------------------------
depart Dublin, 0, 0
blue Dublin, 0, 0
blue West Dublin, 180, 180
blue Castro Valley, 600, 780
blue Bay Fair, 240, 1020
blue San Leandro, 240, 1260
blue Coliseum, 240, 1500
orange Coliseum, 54, 1554
orange Fruitvale, 240, 1794
orange Lake Merritt, 300, 2094
orange 12th Street, 180, 2274
orange 19th Street, 120, 2394
orange MacArthur, 180, 2574
yellow MacArthur, 59, 2633
yellow Rockridge, 240, 2873
yellow Orinda, 300, 3173
yellow Lafayette, 300, 3473
yellow Walnut Creek, 300, 3773
yellow Pleasant Hill, 120, 3893
yellow Concord, 360, 4253
yellow North Concord, 180, 4433
yellow Pittsburg, 360, 4793
yellow Pittsburg Center, 600, 5393
yellow Antioch, 420, 5813
arrive Antioch, 0, 5813


In [23]:
my_neo4j_shortest_path('depart SFO', 'arrive OAK')


--------------------------------
   Total Cost:  3882
   Minutes:  64.7
--------------------------------
depart SFO, 0, 0
yellow SFO, 0, 0
yellow San Bruno, 240, 240
yellow South San Francisco, 240, 480
yellow Colma, 180, 660
yellow Daly City, 240, 900
yellow Balboa Park, 240, 1140
green Balboa Park, 48, 1188
green Glen Park, 120, 1308
green 24th Street Mission, 180, 1488
green 16th Street Mission, 120, 1608
green Civic Center, 180, 1788
green Powell Street, 60, 1848
green Montgomery Street, 120, 1968
green Embarcadero, 60, 2028
green West Oakland, 420, 2448
green Lake Merritt, 360, 2808
green Fruitvale, 300, 3108
green Coliseum, 240, 3348
gray Coliseum, 54, 3402
gray OAK, 480, 3882
arrive OAK, 0, 3882


In [24]:
my_neo4j_shortest_path('depart Antioch', 'arrive Berryessa')


--------------------------------
   Total Cost:  7019
   Minutes:  117.0
--------------------------------
depart Antioch, 0, 0
yellow Antioch, 0, 0
yellow Pittsburg Center, 420, 420
yellow Pittsburg, 600, 1020
yellow North Concord, 360, 1380
yellow Concord, 180, 1560
yellow Pleasant Hill, 360, 1920
yellow Walnut Creek, 120, 2040
yellow Lafayette, 300, 2340
yellow Orinda, 300, 2640
yellow Rockridge, 300, 2940
yellow MacArthur, 240, 3180
orange MacArthur, 59, 3239
orange 19th Street, 180, 3419
orange 12th Street, 120, 3539
orange Lake Merritt, 180, 3719
orange Fruitvale, 300, 4019
orange Coliseum, 240, 4259
orange San Leandro, 240, 4499
orange Bay Fair, 240, 4739
orange Hayward, 240, 4979
orange South Hayward, 240, 5219
orange Union City, 300, 5519
orange Fremont, 300, 5819
orange Warm Springs, 360, 6179
orange Milpitas, 540, 6719
orange Berryessa, 300, 7019
arrive Berryessa, 0, 7019


In [25]:
def my_neo4j_betweeness() :
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)
  
    query = "CALL gds.graph.project('ds_graph', 'Station', {LINK:{properties: 'weight'}})"
    session.run(query)

    query = """

        CALL gds.betweenness.stream('ds_graph')
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
        ORDER BY betweenness DESC

    """
    
    # Parse out the line and station
    # Remove the 'depart' and 'arrive' nodes
    df = my_neo4j_run_query_pandas(query)
    df[['line', 'station']] = df['name'].str.split(n=1, expand = True)
    df = df[df['line'] != 'depart']
    df = df[df['line'] != 'arrive']
    
    return df
   

In [26]:
def my_neo4j_betweeness_RABrandes() :
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)
  
    query = "CALL gds.graph.project('ds_graph', 'Station', {LINK:{properties: 'weight'}})"
    session.run(query)

    query = """

    CALL gds.betweenness.stream('ds_graph', {samplingSize: $sampling_size, samplingSeed: $sampling_seed})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS name, score as betweenness
    ORDER BY betweenness DESC

    """

    sampling_size = 16
    sampling_seed = 0
    
    df = my_neo4j_run_query_pandas(query, sampling_size=sampling_size, sampling_seed=sampling_seed)
    
    # Parse out the line and station
    # Remove the 'depart' and 'arrive' nodes
    df[['line', 'station']] = df['name'].str.split(n=1, expand = True)
    df = df[df['line'] != 'depart']
    df = df[df['line'] != 'arrive']
    
    return df


In [27]:
df = my_neo4j_betweeness()
df.head(20)

,name,betweenness,line,station
0,yellow MacArthur,5999.809223,yellow,MacArthur
1,yellow Rockridge,5509.000000,yellow,Rockridge
2,orange Lake Merritt,5155.831877,orange,Lake Merritt
3,orange 12th Street,5139.715461,orange,12th Street
4,yellow Orinda,4997.000000,yellow,Orinda
5,yellow 19th Street,4820.250748,yellow,19th Street
6,orange Fruitvale,4641.959661,orange,Fruitvale
7,yellow Lafayette,4469.000000,yellow,Lafayette
8,yellow 12th Street,4423.507563,yellow,12th Street
9,orange Coliseum,4306.942363,orange,Coliseum


In [28]:
df.tail(30)

,name,betweenness,line,station
84,blue 24th Street Mission,1388.461823,blue,24th Street Mission
85,green Glen Park,1206.736504,green,Glen Park
86,red El Cerrito Plaza,1144.372316,red,El Cerrito Plaza
87,green Warm Springs,1141.979229,green,Warm Springs
88,orange El Cerrito Plaza,1124.218867,orange,El Cerrito Plaza
89,orange Warm Springs,1111.965825,orange,Warm Springs
90,blue Glen Park,1031.053322,blue,Glen Park
91,yellow San Bruno,972.937915,yellow,San Bruno
92,yellow Pittsburg Center,965.000000,yellow,Pittsburg Center
93,blue West Dublin,965.000000,blue,West Dublin


In [29]:
df = my_neo4j_betweeness_RABrandes()
df.head(30)

,name,betweenness,line,station
0,green West Oakland,554.374176,green,West Oakland
1,green Lake Merritt,493.816752,green,Lake Merritt
2,yellow MacArthur,459.219561,yellow,MacArthur
3,green Coliseum,455.888117,green,Coliseum
4,green Bay Fair,450.616253,green,Bay Fair
5,green San Leandro,435.971670,green,San Leandro
6,orange Lake Merritt,431.633058,orange,Lake Merritt
7,yellow Rockridge,430.000000,yellow,Rockridge
8,green Fruitvale,429.852120,green,Fruitvale
9,yellow 19th Street,402.356035,yellow,19th Street


In [30]:
df.tail(30)

,name,betweenness,line,station
84,yellow South San Francisco,74.793015,yellow,South San Francisco
85,red South San Francisco,71.512750,red,South San Francisco
86,blue Glen Park,67.394551,blue,Glen Park
87,orange Fremont,65.426774,orange,Fremont
88,green Balboa Park,60.685426,green,Balboa Park
89,orange El Cerrito Plaza,59.387470,orange,El Cerrito Plaza
90,red El Cerrito Plaza,56.945531,red,El Cerrito Plaza
91,yellow San Bruno,49.717720,yellow,San Bruno
92,blue Balboa Park,49.298124,blue,Balboa Park
93,yellow Pittsburg Center,48.000000,yellow,Pittsburg Center


In [37]:
def my_neo4j_harmonic() :
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)
  
    query = "CALL gds.graph.project('ds_graph', 'Station', {LINK:{orientation: 'UNDIRECTED'}})"
    session.run(query)
    
    query = """

    CALL gds.closeness.harmonic.stream('ds_graph', {})
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS name, score
    ORDER BY score DESC

    """

    
    df = my_neo4j_run_query_pandas(query)
    
    # Parse out the line and station
    # Remove the 'depart' and 'arrive' nodes
    df[['line', 'station']] = df['name'].str.split(n=1, expand = True)
    df = df[df['line'] != 'depart']
    df = df[df['line'] != 'arrive']
    
    return df


In [39]:
df = my_neo4j_harmonic()
df.head(50)

,name,score,line,station
0,green West Oakland,0.200253,green,West Oakland
1,yellow West Oakland,0.200001,yellow,West Oakland
2,blue West Oakland,0.199824,blue,West Oakland
3,red West Oakland,0.199639,red,West Oakland
4,orange 12th Street,0.194560,orange,12th Street
5,orange Lake Merritt,0.194501,orange,Lake Merritt
6,yellow 12th Street,0.194228,yellow,12th Street
7,green Lake Merritt,0.194012,green,Lake Merritt
8,red 12th Street,0.193799,red,12th Street
9,blue Lake Merritt,0.193495,blue,Lake Merritt


In [53]:
def my_neo4j_louvain() :
    
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)
  
    query = "CALL gds.graph.project('ds_graph', 'Station', { LINK: { orientation: 'UNDIRECTED' } },"
    query = query + "{relationshipProperties: 'weight'})"
        
    session.run(query)
  

    query = """

    CALL gds.louvain.stream('ds_graph')
    YIELD nodeId, communityId, intermediateCommunityIds
    RETURN gds.util.asNode(nodeId).name AS name, communityId
    ORDER BY communityId ASC

    """
    

    df = my_neo4j_run_query_pandas(query)
    
    # Parse out the line and station
    # Remove the 'depart' and 'arrive' nodes
    df[['line', 'station']] = df['name'].str.split(n=1, expand = True)
    df = df[df['line'] != 'depart']
    df = df[df['line'] != 'arrive']
    
    return df


In [65]:
df = my_neo4j_louvain()
df.head(50)

,name,communityId,line,station
0,red Millbrae,50,red,Millbrae
1,red San Bruno,50,red,San Bruno
2,yellow San Bruno,50,yellow,San Bruno
3,red SFO,50,red,SFO
4,yellow SFO,50,yellow,SFO
5,red South San Francisco,50,red,South San Francisco
6,yellow South San Francisco,50,yellow,South San Francisco
17,red Colma,50,red,Colma
18,yellow Colma,50,yellow,Colma
19,green Fremont,55,green,Fremont


In [66]:
macarthur_station_id = df.loc[df['station'] == 'MacArthur']
macarthur_station_id


,name,communityId,line,station
77,orange MacArthur,181,orange,MacArthur
78,red MacArthur,181,red,MacArthur
79,yellow MacArthur,181,yellow,MacArthur


In [57]:
coliseum_community_id = df.loc[df['station'] == 'Coliseum']
coliseum_community_id

,name,communityId,line,station
156,blue Coliseum,197,blue,Coliseum
157,gray Coliseum,197,gray,Coliseum
158,green Coliseum,197,green,Coliseum
159,orange Coliseum,197,orange,Coliseum


In [68]:
macarthur_community_df = df.loc[df['communityId'] == 181]
macarthur_community_df

,name,communityId,line,station
77,orange MacArthur,181,orange,MacArthur
78,red MacArthur,181,red,MacArthur
79,yellow MacArthur,181,yellow,MacArthur
86,orange 12th Street,181,orange,12th Street
87,red 12th Street,181,red,12th Street
88,yellow 12th Street,181,yellow,12th Street
89,orange 19th Street,181,orange,19th Street
90,red 19th Street,181,red,19th Street
91,yellow 19th Street,181,yellow,19th Street


In [71]:
coliseum_community_df = df.loc[df['communityId'] == 197]
coliseum_community_df

,name,communityId,line,station
141,blue Fruitvale,197,blue,Fruitvale
142,green Fruitvale,197,green,Fruitvale
143,orange Fruitvale,197,orange,Fruitvale
144,blue Lake Merritt,197,blue,Lake Merritt
145,green Lake Merritt,197,green,Lake Merritt
146,orange Lake Merritt,197,orange,Lake Merritt
147,gray OAK,197,gray,OAK
156,blue Coliseum,197,blue,Coliseum
157,gray Coliseum,197,gray,Coliseum
158,green Coliseum,197,green,Coliseum


In [74]:
bayfair_station_id = df.loc[df['station'] == 'Bay Fair']
bayfair_station_id

,name,communityId,line,station
124,blue Bay Fair,191,blue,Bay Fair
125,green Bay Fair,191,green,Bay Fair
126,orange Bay Fair,191,orange,Bay Fair


In [76]:
bayfair_community_df = df.loc[df['communityId'] == 191]
bayfair_community_df

,name,communityId,line,station
110,blue San Leandro,191,blue,San Leandro
111,green San Leandro,191,green,San Leandro
112,orange San Leandro,191,orange,San Leandro
113,blue West Dublin,191,blue,West Dublin
124,blue Bay Fair,191,blue,Bay Fair
125,green Bay Fair,191,green,Bay Fair
126,orange Bay Fair,191,orange,Bay Fair
127,blue Castro Valley,191,blue,Castro Valley
128,blue Dublin,191,blue,Dublin


In [77]:
dalycity_station_id = df.loc[df['station'] == 'Daly City']
dalycity_station_id

,name,communityId,line,station
106,blue Daly City,182,blue,Daly City
107,green Daly City,182,green,Daly City
108,red Daly City,182,red,Daly City
109,yellow Daly City,182,yellow,Daly City


In [79]:
dalycity_community_df = df.loc[df['communityId'] == 182]
dalycity_community_df

,name,communityId,line,station
92,blue Glen Park,182,blue,Glen Park
93,green Glen Park,182,green,Glen Park
94,red Glen Park,182,red,Glen Park
95,yellow Glen Park,182,yellow,Glen Park
102,blue Balboa Park,182,blue,Balboa Park
103,green Balboa Park,182,green,Balboa Park
104,red Balboa Park,182,red,Balboa Park
105,yellow Balboa Park,182,yellow,Balboa Park
106,blue Daly City,182,blue,Daly City
107,green Daly City,182,green,Daly City
